In [1]:
!conda install -y pyvips

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [2]:
BAD_IMAGE_IDS = ['5adc4c_0', '7b9aaa_0', 'bb06a5_0', 'e26a04_0', '280c26_0'] + \
                ['4ae44b_0', '53e66f_0', '7c2c2f_0', '74a450_1']

BLOCK_SIZE = 28
BLOCKS_PER_CROP = 8
CROP_SIZE = BLOCK_SIZE * BLOCKS_PER_CROP
BLOCK_THR = 90
CROP_THR = 0.6
MAX_CROPS_PER_IMAGE = 20
IMAGES_PER_SAMPLE = 4
EPOCHS_NUM = 10
SCALE_FACTOR = 24

In [3]:
import gc
import os
from time import time
from typing import List, Tuple

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import pyvips
import cv2


class DataPreparation:
    def __init__(self, visualize: bool = False, seed: int = 42):
        self.visualize = visualize
        self.seed = seed

        train_metadata = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
        train_metadata = list(zip(
            train_metadata['image_id'].tolist(),
            train_metadata['label'].tolist(),
            train_metadata['center_id'].tolist(),
        ))
        self.train = self._filter_bad_images(train_metadata)
        self.all_center_ids = sorted(list({center_id for _, _, center_id in self.train}))

        other_metadata = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/other.csv').query('label == \'Other\'')
        other_metadata = list(zip(
            other_metadata['image_id'].tolist(),
            ['LAA' for _ in range(other_metadata.shape[0])],
            [-1 for _ in range(other_metadata.shape[0])],
        ))
        self.other = self._filter_bad_images(other_metadata)

    @staticmethod
    def _filter_bad_images(data: List[Tuple]) -> List[Tuple]:
        return [
            (image_id, label, center_id)
            for image_id, label, center_id in data
            if image_id not in BAD_IMAGE_IDS
        ]

    @staticmethod
    def _add_rect_to_numpy(image: np.ndarray, x: int, y: int, size: int, thickness: int) -> None:
        image[x:x + size, y:y + thickness] = (0, 0, 0)
        image[x:x + thickness, y:y + size] = (0, 0, 0)
        image[x:x + size, y + size:y + size + thickness] = (0, 0, 0)
        image[x + size:x + size + thickness, y:y + size] = (0, 0, 0)

    @staticmethod
    def _get_blocks_map(image: np.ndarray) -> np.ndarray:
        pixels_diff = np.sum((image[:-1, :, :] - image[1:, :, :]) ** 2, axis=2)
        pixels_diff = np.cumsum(np.cumsum(pixels_diff, axis=0), axis=1)
        blocks_map = np.zeros((
            (image.shape[0] + BLOCK_SIZE - 1) // BLOCK_SIZE,
            (image.shape[1] + BLOCK_SIZE - 1) // BLOCK_SIZE,
        ))
        for x in range(0, pixels_diff.shape[0], BLOCK_SIZE):
            for y in range(0, pixels_diff.shape[1], BLOCK_SIZE):
                nx = min(x + BLOCK_SIZE, pixels_diff.shape[0])
                ny = min(y + BLOCK_SIZE, pixels_diff.shape[1])
                block_sum = int(pixels_diff[nx - 1, ny - 1])
                if x:
                    block_sum -= int(pixels_diff[x - 1, ny - 1])
                if y:
                    block_sum -= int(pixels_diff[nx - 1, y - 1])
                if x and y:
                    block_sum += int(pixels_diff[x - 1, y - 1])
                blocks_map[x // BLOCK_SIZE][y // BLOCK_SIZE] = \
                    (block_sum / BLOCK_SIZE / BLOCK_SIZE) > BLOCK_THR
        return blocks_map

    def _generate_crops_positions(
            self,
            image: np.ndarray,
            crop_thr: float,
    ) -> Tuple[List[Tuple[int, int]], np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        blocks_map = self._get_blocks_map(image)

        if self.visualize:
            for i in range(blocks_map.shape[0]):
                for j in range(blocks_map.shape[1]):
                    if blocks_map[i][j]:
                        self._add_rect_to_numpy(
                            image,
                            i * BLOCK_SIZE,
                            j * BLOCK_SIZE,
                            BLOCK_SIZE,
                            1,
                        )

        good_crops_starts = []
        for x in range(0, image.shape[0] - CROP_SIZE + 1, BLOCK_SIZE):
            for y in range(0, image.shape[1] - CROP_SIZE + 1, BLOCK_SIZE):
                _x, _y = x // BLOCK_SIZE, y // BLOCK_SIZE
                crop_sum = blocks_map[_x:_x + BLOCKS_PER_CROP, _y:_y + BLOCKS_PER_CROP].sum()
                if crop_sum > BLOCKS_PER_CROP * BLOCKS_PER_CROP * crop_thr:
                    good_crops_starts.append((x, y))

        if self.visualize:
            for x, y in good_crops_starts:
                self._add_rect_to_numpy(image, x, y, CROP_SIZE, 1)

        return good_crops_starts

    @staticmethod
    def _process_crop(crop: np.ndarray) -> np.ndarray:
        return crop

    def _create_crops(
        self,
        image: np.ndarray,
        crops_starts: List[Tuple[int]],
    ) -> List[np.ndarray]:
        return [
            Image.fromarray(
                self._process_crop(
                    image[x:x + CROP_SIZE, y:y + CROP_SIZE],
                )
            )
            for x, y in crops_starts
        ]

    @staticmethod
    def _get_unique_crops(crop_starts: List[Tuple[int, int]], order) -> List[Tuple[int, int]]:
        def inter_size_1d(a: int, b: int, c: int, d: int) -> int:
            return max(0, min(b, d) - max(a, c))

        def inter_size_2d(crop_start_1: Tuple[int, int], crop_start_2: Tuple[int, int]) -> int:
            return inter_size_1d(
                crop_start_1[0], crop_start_1[0] + CROP_SIZE,
                crop_start_2[0], crop_start_2[0] + CROP_SIZE,
            ) * inter_size_1d(
                crop_start_1[1], crop_start_1[1] + CROP_SIZE,
                crop_start_2[1], crop_start_2[1] + CROP_SIZE,
            )

        crop_starts_sorted = sorted(crop_starts, key=order)
        final_crop_starts = []
        for crop_start in crop_starts_sorted:
            if any(
                    inter_size_2d(crop_start, crop_start_prev) > CROP_SIZE * CROP_SIZE // 2
                    for crop_start_prev in final_crop_starts
            ):
                continue
            final_crop_starts.append(crop_start)
        return final_crop_starts
    
    @staticmethod
    def _read_and_resize_image(image_id: str, base_image_path: str) -> np.ndarray:       
        image_path = os.path.join(base_image_path, f'{image_id}.tif')
        image = pyvips.Image.new_from_file(image_path, access='sequential')
        return image.resize(1.0 / SCALE_FACTOR).numpy()
    
    def prepare_crops(
            self,
            image_ids: List[int],
            base_image_path: str,
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]], List[Tuple[np.ndarray, np.ndarray]]]:
        np.random.seed(self.seed)
        image_crops = []
        image_crops_indices = []
        for image_id in tqdm(image_ids):
            start_time = time()
            image = self._read_and_resize_image(image_id, base_image_path)
            gc.collect()
            print(f'Rescaling done in {time() - start_time} seconds. Image shape is {image.shape}')
            found_flag = False
            for crop_thr in np.arange(CROP_THR, -0.1, -0.1):
                good_crops_starts = self._generate_crops_positions(image, crop_thr)
                if len(good_crops_starts) < IMAGES_PER_SAMPLE:
                    print('Bad image', image_id, 'crop_thr', crop_thr, 'only', len(good_crops_starts))
                    continue

                good_crops_starts_unique = []
                for order in [
                    lambda x: (x[0], x[1]),
                    lambda x: (-x[0], -x[1]),
                ]:
                    good_crops_starts_unique.extend(self._get_unique_crops(good_crops_starts, order))
                good_crops_starts_unique = list(set(good_crops_starts_unique))

                if len(good_crops_starts_unique) < IMAGES_PER_SAMPLE:
                    print('Bad image', image_id, 'crop_thr', crop_thr, 'only', len(good_crops_starts_unique))
                    continue

                good_crops_starts_sample_ids = np.random.choice(
                    list(range(len(good_crops_starts_unique))),
                    min(len(good_crops_starts_unique), MAX_CROPS_PER_IMAGE),
                    replace=False,
                )
                good_crops_starts_sample = np.array(good_crops_starts_unique)[good_crops_starts_sample_ids]
                image_crops_indices.append(good_crops_starts_sample)
                image_crops.append(self._create_crops(image, good_crops_starts_sample))
                found_flag = True
                break
            if not found_flag:
                image_crops_indices.append([])
                image_crops.append([])
                print('No crops was found')
            print(f'Done {image_id} in {time() - start_time} seconds')
        gc.collect()
        return image_crops, image_crops_indices

    def process_train(
            self
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]]]:
        return self.prepare_crops(
            [image_id for image_id, _, _ in self.train],
            '/kaggle/input/mayo-clinic-strip-ai/train/',
        )

    def process_other(
            self
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]]]:
        return self.prepare_crops(
            [image_id for image_id, _, _ in self.other],
            '/kaggle/input/mayo-clinic-strip-ai/other/',
        )

In [4]:
from __future__ import print_function, division

import os
import pickle
import sys
from collections import Counter

import cv2
import numpy as np
import ssl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from PIL import Image
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from torchvision import transforms


ssl._create_default_https_context = ssl._create_unverified_context
cudnn.benchmark = True


DUMPED_DATALOADER_PATH = '/kaggle/working/data_loaders.pkl'
DUMPED_DATALOADER_OTHER_PATH = '/kaggle/working/data_loaders_other.pkl'


def get_sub_data(data, image_crops, image_crops_indices, sample_ids):
    return [data[i][0] for i in sample_ids], \
        [data[i][1] for i in sample_ids], \
        [data[i][2] for i in sample_ids], \
        [image_crops[i] for i in sample_ids], \
        [image_crops_indices[i] for i in sample_ids]


data_prep = DataPreparation()

image_crops, image_crops_indices = data_prep.process_train()
with open(DUMPED_DATALOADER_PATH, 'wb') as file:
    pickle.dump([image_crops, image_crops_indices], file)

image_crops_other, image_crops_indices_other = data_prep.process_other()
with open(DUMPED_DATALOADER_OTHER_PATH, 'wb') as file:
    pickle.dump([image_crops_other, image_crops_indices_other], file)

  0%|          | 0/749 [00:00<?, ?it/s]

Rescaling done in 56.89508581161499 seconds. Image shape is (2533, 1417, 3)


  0%|          | 1/749 [00:57<11:53:44, 57.25s/it]

Done 006388_0 in 57.25131821632385 seconds


  0%|          | 2/749 [01:00<5:17:31, 25.50s/it] 

...

Rescaling done in 5.432465314865112 seconds. Image shape is (1346, 403, 3)
Bad image eb5e78_0 crop_thr 0.6 only 0
Bad image eb5e78_0 crop_thr 0.5 only 0
Done eb5e78_0 in 5.524275302886963 seconds


 98%|█████████▊| 64/65 [42:21<00:52, 52.65s/it]

Rescaling done in 7.592771768569946 seconds. Image shape is (1220, 665, 3)
Bad image efead4_0 crop_thr 0.6 only 0
Bad image efead4_0 crop_thr 0.5 only 0
Bad image efead4_0 crop_thr 0.4 only 0
Done efead4_0 in 7.7815234661102295 seconds


100%|██████████| 65/65 [42:26<00:00, 39.18s/it]

Rescaling done in 5.0498740673065186 seconds. Image shape is (591, 478, 3)
Done f2b07d_0 in 5.079759359359741 seconds
